<a href="https://colab.research.google.com/github/Ezhil2318/Pandas_Practice/blob/main/pandasPractice_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df_sales = pd.DataFrame({
    'TransactionID': [101, 102, 103, 104, 105],
    'CustomerID': ['C1', 'C2', 'C1', 'C3', 'C4'],
    'Product': ['Laptop', 'Tablet', 'Laptop', 'Phone', 'Tablet'],
    'Quantity': [1, 2, 1, 1, 2],
    'Amount': [1200, 600, 1200, 800, 1200],
    'Date': pd.to_datetime(['2023-01-01', '2023-01-02', '2023-01-05', '2023-01-06', '2023-01-07'])
})

In [ ]:
df_sales.groupby('CustomerID')['Amount'].sum().reset_index(name='Total_spent')

,CustomerID,Total_spent
0,C1,2400
1,C2,600
2,C3,800
3,C4,1200


In [ ]:
df_sales.pivot_table(index='Product',columns='CustomerID',values='Quantity',fill_value=0)

CustomerID,C1,C2,C3,C4
Product,,,,
Laptop,1.0,0.0,0.0,0.0
Phone,0.0,0.0,1.0,0.0
Tablet,0.0,2.0,0.0,2.0


In [ ]:
x = df_sales.assign(runningTotal = df_sales.groupby('CustomerID')['Amount'].transform('sum'))
x

,TransactionID,CustomerID,Product,Quantity,Amount,Date,runningTotal
0,101,C1,Laptop,1,1200,2023-01-01,2400
1,102,C2,Tablet,2,600,2023-01-02,600
2,103,C1,Laptop,1,1200,2023-01-05,2400
3,104,C3,Phone,1,800,2023-01-06,800
4,105,C4,Tablet,2,1200,2023-01-07,1200


In [ ]:
x.assign(rank = x['runningTotal'].rank(method='dense',ascending=False))

,TransactionID,CustomerID,Product,Quantity,Amount,Date,runningTotal,rank
0,101,C1,Laptop,1,1200,2023-01-01,2400,1.0
1,102,C2,Tablet,2,600,2023-01-02,600,4.0
2,103,C1,Laptop,1,1200,2023-01-05,2400,1.0
3,104,C3,Phone,1,800,2023-01-06,800,3.0
4,105,C4,Tablet,2,1200,2023-01-07,1200,2.0


In [ ]:
df_inventory = pd.DataFrame({
    'Product': ['Laptop', 'Tablet', 'Phone', 'Monitor', 'Keyboard'],
    'Stock': [10, 25, 15, 7, 40],
    'ReorderLevel': [5, 10, 5, 3, 20],
    'SupplierID': ['S1', 'S2', 'S1', 'S3', 'S2']
})

df_suppliers = pd.DataFrame({
    'SupplierID': ['S1', 'S2', 'S3'],
    'SupplierName': ['TechSource', 'GadgetFlow', 'ScreenMart'],
    'Country': ['USA', 'China', 'Germany']
})

In [ ]:
supInv = df_suppliers.merge(df_inventory,on='SupplierID',how='inner')
supInv.iloc[:,:-2]

,SupplierID,SupplierName,Country,Product
0,S1,TechSource,USA,Laptop
1,S1,TechSource,USA,Phone
2,S2,GadgetFlow,China,Tablet
3,S2,GadgetFlow,China,Keyboard
4,S3,ScreenMart,Germany,Monitor


In [ ]:
df_inventory.assign(flag = (df_inventory.ReorderLevel<=5)).replace({False:'High',True:'Low'})

,Product,Stock,ReorderLevel,SupplierID,flag
0,Laptop,10,5,S1,Low
1,Tablet,25,10,S2,High
2,Phone,15,5,S1,Low
3,Monitor,7,3,S3,Low
4,Keyboard,40,20,S2,High


In [ ]:
supInv.groupby('SupplierName')['Product'].nunique().reset_index(name='Product_count')

,SupplierName,Product_count
0,GadgetFlow,2
1,ScreenMart,1
2,TechSource,2


In [ ]:
df_feedback = pd.DataFrame({
    'CustomerID': ['C1', 'C2', 'C1', None, 'C4', 'C3'],
    'Rating': [5, 4, None, 3, 2, 4],
    'Comment': ['Excellent', 'Good', '', 'Average', None, 'Nice']
})

In [ ]:
df_feedback.replace('',None,inplace=True)

In [ ]:
def comment(row):
  if pd.isna(row['Comment']):
    if row['Rating']<3 or pd.isna(row['Rating']):
      return 'Poor'
    elif row['Rating'] == 3:
      return 'Average'
    elif row['Rating'] == 4:
      return 'Good'
    else:
      return 'Excellent'
  else:
    return row['Comment']
df_feedback['Comment'] = df_feedback.apply(comment,axis=1)

In [ ]:
df_feedback

,CustomerID,Rating,Comment
0,C1,5.0,Excellent
1,C2,4.0,Good
2,C1,NaN,Poor
3,None,3.0,Average
4,C4,2.0,Poor
5,C3,4.0,Nice


In [ ]:
df_feedback.groupby('CustomerID')['Rating'].mean().reset_index(name='Avg_Rating')

,CustomerID,Avg_Rating
0,C1,5.0
1,C2,4.0
2,C3,4.0
3,C4,2.0


In [ ]:
df_feedback[(df_feedback.groupby('CustomerID')['Comment'].transform('count')>1)&(df_feedback.Rating.isna())]

,CustomerID,Rating,Comment
2,C1,NaN,Poor


In [ ]:
merged1 = df_sales.merge(df_feedback,on='CustomerID',how='left')
merged1

,TransactionID,CustomerID,Product,Quantity,Amount,Date,Rating,Comment
0,101,C1,Laptop,1,1200,2023-01-01,5.0,Excellent
1,101,C1,Laptop,1,1200,2023-01-01,NaN,Poor
2,102,C2,Tablet,2,600,2023-01-02,4.0,Good
3,103,C1,Laptop,1,1200,2023-01-05,5.0,Excellent
4,103,C1,Laptop,1,1200,2023-01-05,NaN,Poor
5,104,C3,Phone,1,800,2023-01-06,4.0,Nice
6,105,C4,Tablet,2,1200,2023-01-07,2.0,Poor


In [ ]:
merged2 = df_suppliers.merge(df_inventory,on='SupplierID',how='inner')
merged2

,SupplierID,SupplierName,Country,Product,Stock,ReorderLevel
0,S1,TechSource,USA,Laptop,10,5
1,S1,TechSource,USA,Phone,15,5
2,S2,GadgetFlow,China,Tablet,25,10
3,S2,GadgetFlow,China,Keyboard,40,20
4,S3,ScreenMart,Germany,Monitor,7,3


In [ ]:
df_inventory['Flagged'] = (df_inventory.ReorderLevel<=3)
df_inventory

,Product,Stock,ReorderLevel,SupplierID,Flagged
0,Laptop,10,5,S1,False
1,Tablet,25,10,S2,False
2,Phone,15,5,S1,False
3,Monitor,7,3,S3,True
4,Keyboard,40,20,S2,False


In [ ]:
merged = df_inventory.merge(df_sales,on='Product',how='left')

In [ ]:
x = merged[merged.Flagged==True].groupby('CustomerID')['Amount'].sum().reset_index(name='Total_spent').rank(method='dense')
x = x.merge(df_feedback,on='CustomerID',how='inner')
x['Rating'] = x['Rating'].fillna(x.groupby('CustomerID')['Rating'].transform('mean'))
x['Flag'] = (x.Rating<3)
x['Flag'] = x['Flag'].replace({True:'Critical',False:'Not Critical'})

In [ ]:

df_returns = pd.DataFrame({
    'OrderID': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    'Product': ['X1', 'X2', 'X1', 'X3', 'X4', 'X1', 'X2', 'X1', 'X3', 'X2'],
    'ReturnReason': ['Damaged', 'Expired', 'Damaged', 'Size Issue', 'Damaged',
                     'Expired', 'Damaged', 'Damaged', 'Expired', 'Expired']
})
